### Deduction List
This workbook is going to generate the deduction list of all users
It will have two parts:
1. clustering pretrain & location authority score emerging
2. location filtering

In [1]:
import pandas as pd
import numpy as np

### 1. clustering pretrain & location authority score emerging

In [2]:
# import cluster pretrain score
import pickle
def load_pickle(name):
    with open(name, 'rb') as f:
        return pickle.load(f, encoding='latin1')


In [3]:
cluster_score = load_pickle('clusters_score.pkl')
cluster_score.shape

(1083, 200)

In [4]:
# convert to pandas df
cluster_df = pd.DataFrame(cluster_score)
cluster_df = cluster_df.reset_index()
cluster_df['index'] += 1
cluster_df.rename(columns = {'index': 'user_id'}, inplace=True)
cluster_df

,user_id,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,1,1.002286e-07,-2.033700,-0.525726,-0.901425,-0.752404,-0.593636,-2.540184,-3.178983,-2.082429,...,-1.597055,-0.729059,-4.574325,-0.009790,0.715054,-6.021991,-4.024022,-1.042716,-0.051618,-0.229525
1,2,1.407860e-07,-2.149609,-1.826800,0.335773,-1.808471,-3.071292,-1.550357,-3.887939,-2.907891,...,-8.294293,-3.374946,-7.931877,-2.598221,-0.849538,-10.953440,-8.887760,-3.013627,-6.894687,0.687203
2,3,1.743462e-07,-1.429179,-3.534334,-1.098754,-1.909353,-3.058600,-2.813250,-4.263647,-2.882062,...,-2.318683,-2.702227,-6.319556,-1.317822,-0.523653,-9.578644,-7.175983,-4.606969,-5.020787,0.191002
3,4,-5.045619e-09,-3.311185,-5.600921,-1.128353,-2.549431,-6.697231,-0.633725,-4.024603,-3.938465,...,-6.530472,-4.593059,-9.502889,-3.626084,-3.061975,-13.302953,-12.777984,-6.703866,-11.771523,-0.213975
4,5,-2.850616e-07,-5.266183,-5.777240,-2.329710,-1.783113,-4.547246,-1.445422,-4.029823,-4.325774,...,-6.447091,-2.975491,-13.447259,-5.525450,-3.447876,-15.617734,-11.232311,-5.414327,-9.777754,-1.765407
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078,1079,-2.836634e-07,-2.939964,-7.084353,-1.688998,-2.126004,-6.128013,-6.392339,-5.023687,-3.858847,...,-7.160411,-2.394122,-10.744199,-1.678887,-2.012657,-14.562045,-10.297093,-4.786314,-5.921826,-0.990262
1079,1080,-1.879851e-07,-5.900034,-3.862690,-0.953542,-1.095514,-2.348069,-3.569865,-5.085397,-3.995157,...,-2.917059,-1.746640,-8.579536,-1.032523,-0.696496,-8.551124,-6.700329,-4.508793,-1.498734,-1.209552
1080,1081,-3.612553e-07,-6.408042,-6.373474,-0.896908,0.706463,-5.474352,-2.813390,-4.611372,-3.917036,...,-4.018150,-2.541934,-11.714668,-2.576519,-1.914982,-11.990095,-9.599609,-5.318696,-4.866495,-0.039076
1081,1082,-8.421253e-08,-1.063055,-5.605690,-0.608649,1.760613,-3.207598,-3.164924,-4.026205,-1.432225,...,-3.223836,-2.365294,-11.240274,-1.728866,0.009644,-13.208838,-10.328435,-3.688047,-6.483699,1.745486


In [5]:
# melt the table
cluster_df = pd.melt(cluster_df, 
               id_vars = 'user_id',
               value_vars = list(cluster_df.columns[1:]),
               var_name = 'clusters',
               value_name = 'score' 

               )
cluster_df = cluster_df.sort_values(by = ['user_id', 'clusters'], ascending = True)
cluster_df['clusters'] += 1

#### deal with the negative value

In [6]:
cluster_df['score'] = cluster_df['score'] - cluster_df['score'].min()
cluster_df

,user_id,clusters,score
0,1,1,22.013010
1083,1,2,19.979310
2166,1,3,21.487284
3249,1,4,21.111585
4332,1,5,21.260606
...,...,...,...
212267,1083,196,14.083699
213350,1083,197,16.408518
214433,1083,198,20.733309
215516,1083,199,21.003180


In [7]:
# import cluster
cluster = pd.read_csv("poi_cluster.csv")
cluster.nunique()

poi_id      9989
clusters     200
dtype: int64

In [8]:
# create poi candidate set, the size is: num of poi x num of users
candidate = cluster_df.merge(cluster, on = 'clusters', how = 'outer')
candidate.head()

,user_id,clusters,score,poi_id
0,1,1,22.01301,3fd66200f964a52011e91ee3
1,1,1,22.01301,3fd66200f964a5201ae61ee3
2,1,1,22.01301,3fd66200f964a5201fee1ee3
3,1,1,22.01301,3fd66200f964a52022e41ee3
4,1,1,22.01301,3fd66200f964a52025eb1ee3


In [9]:
# merge with location authority scores
loc_auth = pd.read_csv('location_auth_score.csv')
loc_auth['score'] = loc_auth['score'] * 100

In [10]:
loc_auth.max()

poi_encode                        9988
score                         0.230907
poi_id        5116fef9e4b066680e7fab07
dtype: object

In [11]:
# multiply
candidate = candidate.merge(loc_auth, on = 'poi_id', how = 'left')
candidate['union_score'] = candidate['score_x'] * candidate['score_y']
candidate = candidate[['user_id', 'poi_id', 'union_score']]
candidate

,user_id,poi_id,union_score
0,1,3fd66200f964a52011e91ee3,0.165507
1,1,3fd66200f964a5201ae61ee3,0.164581
2,1,3fd66200f964a5201fee1ee3,0.179501
3,1,3fd66200f964a52022e41ee3,0.173020
4,1,3fd66200f964a52025eb1ee3,0.169938
...,...,...,...
10818082,1083,4fa84566e4b0b1248794c127,0.145816
10818083,1083,4fad7debe4b08e3811784f2d,0.169947
10818084,1083,4fe9e294e4b0faf491da7927,0.159129
10818085,1083,50b544a2e4b04756f5eb2b0e,0.182922


### 2. location filtering
this part is going to filter the POIs based on the users current location 

In [12]:
# distance function
from math import radians, cos, sin, asin, sqrt
def geodistance(lng1, lat1, lng2, lat2):

    lng1, lat1, lng2, lat2 = map(radians, [float(lng1), float(lat1), float(lng2), float(lat2)])
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2 
    distance=2*asin(sqrt(a))*6371*1000
    distance=round(distance/1000,3)
    return distance

In [13]:
dir = 'E:\\Sebnewrepo/Rec_sys_lab/paper1_experiment/'
checkin_file = 'ny_ordered.csv'
df = pd.read_csv(dir + checkin_file)
df.head()

,user_id,poi_id,poi_category_id,poi_category_name,latitude,longitude,time_offset,UTC_time,datetime
0,1,4abc1f51f964a520798620e3,4bf58dd8d48988d1ce941735,Seafood Restaurant,40.781558,-73.975792,-240,Wed Apr 04 23:31:31 +0000 2012,2012-04-04 23:31:31
1,1,4d4ac10da0ef54814b6ffff6,4bf58dd8d48988d157941735,American Restaurant,40.784018,-73.974524,-240,Sat Apr 07 17:42:24 +0000 2012,2012-04-07 17:42:24
2,1,4db44994cda1c57c82583709,4bf58dd8d48988d1f1931735,General Entertainment,40.739398,-73.993210,-240,Sun Apr 08 18:20:29 +0000 2012,2012-04-08 18:20:29
3,1,4a541923f964a52008b31fe3,4bf58dd8d48988d14e941735,American Restaurant,40.785677,-73.976498,-240,Sun Apr 08 20:02:10 +0000 2012,2012-04-08 20:02:10
4,1,40f1d480f964a5205b0a1fe3,4bf58dd8d48988d143941735,Breakfast Spot,40.719929,-74.008532,-240,Mon Apr 09 16:20:52 +0000 2012,2012-04-09 16:20:52


In [14]:
# get the current position
df = df.reset_index()
df['rank'] = df.groupby('user_id')['index'].rank(ascending = False)
current_position = df[df['rank'] < 2]
current_position = current_position[['user_id', 'latitude', 'longitude']]
current_position.head()

,user_id,latitude,longitude
69,1,40.773839,-73.871220
169,2,40.567089,-73.865461
254,3,40.743597,-74.003327
385,4,40.816816,-73.941393
477,5,40.728917,-74.041350


In [15]:
# extract all location cordinates
poi_cor = df[['poi_id','latitude','longitude']]
poi_cor = poi_cor.drop_duplicates(subset = ['poi_id'])
# merge with candidate set
candidate = candidate.merge(poi_cor, on = 'poi_id', how = 'left')
candidate.head(10)

,user_id,poi_id,union_score,latitude,longitude
0,1,3fd66200f964a52011e91ee3,0.165507,40.730641,-74.002705
1,1,3fd66200f964a5201ae61ee3,0.164581,40.728898,-73.998128
2,1,3fd66200f964a5201fee1ee3,0.179501,40.672797,-73.983208
3,1,3fd66200f964a52022e41ee3,0.173020,40.718152,-73.990002
4,1,3fd66200f964a52025eb1ee3,0.169938,40.728988,-73.988404
5,1,3fd66200f964a52034eb1ee3,0.175945,40.719455,-73.996361
6,1,3fd66200f964a52039e31ee3,0.180627,40.722555,-73.987872
7,1,3fd66200f964a52041eb1ee3,0.178409,40.755201,-73.971022
8,1,3fd66200f964a5204de41ee3,0.174893,40.726960,-73.989061
9,1,3fd66200f964a5204ef11ee3,0.177565,40.746369,-73.899219


In [16]:
candidate = candidate.merge(current_position, on = 'user_id', how = 'left')
candidate.head()

,user_id,poi_id,union_score,latitude_x,longitude_x,latitude_y,longitude_y
0,1,3fd66200f964a52011e91ee3,0.165507,40.730641,-74.002705,40.773839,-73.87122
1,1,3fd66200f964a5201ae61ee3,0.164581,40.728898,-73.998128,40.773839,-73.87122
2,1,3fd66200f964a5201fee1ee3,0.179501,40.672797,-73.983208,40.773839,-73.87122
3,1,3fd66200f964a52022e41ee3,0.173020,40.718152,-73.990002,40.773839,-73.87122
4,1,3fd66200f964a52025eb1ee3,0.169938,40.728988,-73.988404,40.773839,-73.87122


In [17]:
# distance function
from math import radians, cos, sin, asin, sqrt
def geodistance1(df):

    lng1, lat1, lng2, lat2 = map(radians, [float(df['longitude_x']), float(df['latitude_x']), float(df['longitude_y']), float(df['latitude_y'])])
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2 
    distance=2*asin(sqrt(a))*6371*1000
    distance=round(distance/1000,3)
    return distance

In [18]:
candidate['distance'] = candidate.apply(geodistance1, axis = 1)

In [19]:
candidate['rank'] = candidate.groupby('user_id')['union_score'].rank(ascending = False)


In [20]:
candidate[candidate['user_id'] == 2].sort_values(by = ['union_score'])

,user_id,poi_id,union_score,latitude_x,longitude_x,latitude_y,longitude_y,distance,rank
6751442,2,4b256ab7f964a5209e7124e3,0.088580,40.785864,-73.950944,40.567089,-73.865461,25.372,9989.0
6751456,2,4f2492eae4b0085e4a289f30,0.091719,40.686504,-73.883070,40.567089,-73.865461,13.361,9988.0
6751445,2,4bfd2d30f7c82d7f58c08d04,0.095579,40.632319,-73.918269,40.567089,-73.865461,8.514,9987.0
5851462,2,4b864e7af964a520738531e3,0.099235,40.967786,-74.073690,40.567089,-73.865461,47.882,9986.0
10570086,2,4c0e4252336220a1a7ddcb77,0.100362,40.967839,-74.073782,40.567089,-73.865461,47.890,9985.0
...,...,...,...,...,...,...,...,...,...
6117973,2,4840fe6bf964a52030501fe3,2.380706,40.757221,-73.991547,40.567089,-73.865461,23.666,5.0
6118006,2,4ace6c89f964a52078d020e3,2.879424,40.773839,-73.871220,40.567089,-73.865461,22.995,4.0
6117970,2,43a52546f964a520532c1fe3,3.064283,40.645089,-73.784523,40.567089,-73.865461,11.041,3.0
6117965,2,42829c80f964a5206a221fe3,3.296710,40.752806,-73.977175,40.567089,-73.865461,22.699,2.0


In [21]:
candidate_output = candidate[['user_id','poi_id','rank','distance']]
candidate_output.to_csv('E:/Sebnewrepo/Data/paper1_data/ny_candidate.csv')